In [1]:
import nba_api
import pandas as pd
import warnings
from tqdm import tqdm
from time import sleep

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [2]:
from nba_api.stats.endpoints.teamgamelogs import TeamGameLogs
from nba_api.stats.endpoints.teamdashboardbygeneralsplits import TeamDashboardByGeneralSplits

In [3]:
logs1 = TeamGameLogs(season_nullable='2015-16').get_data_frames()[0]
sleep(1)
logs2 = TeamGameLogs(season_nullable='2016-17').get_data_frames()[0]
sleep(1)
logs3 = TeamGameLogs(season_nullable='2017-18').get_data_frames()[0]
sleep(1)
logs4 = TeamGameLogs(season_nullable='2018-19').get_data_frames()[0]
sleep(1)
logs5 = TeamGameLogs(season_nullable='2019-20').get_data_frames()[0]
sleep(1)
logs6 = TeamGameLogs(season_nullable='2020-21').get_data_frames()[0]
sleep(1)
logs7 = TeamGameLogs(season_nullable='2021-22').get_data_frames()[0]

In [4]:
logs = pd.concat([logs1, logs2, logs3, logs4, logs5, logs6, logs7], axis=0)
logs.drop(logs.iloc[:, 30:], inplace=True, axis=1)
logs.tail()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS
1331,2021-22,1610612753,ORL,Orlando Magic,0022100010,2021-10-20T00:00:00,ORL @ SAS,L,48.0,35,86,0.407,9,29,0.310,18,23,0.783,13,31,44,17,16.0,6,6,3,20,22,97,-26.0
1332,2021-22,1610612749,MIL,Milwaukee Bucks,0022100001,2021-10-19T00:00:00,MIL vs. BKN,W,48.0,48,105,0.457,17,45,0.378,14,18,0.778,13,41,54,25,8.0,8,9,9,19,17,127,23.0
1333,2021-22,1610612751,BKN,Brooklyn Nets,0022100001,2021-10-19T00:00:00,BKN @ MIL,L,48.0,37,84,0.440,17,32,0.531,13,23,0.565,5,39,44,19,13.0,3,9,9,17,19,104,-23.0
1334,2021-22,1610612747,LAL,Los Angeles Lakers,0022100002,2021-10-19T00:00:00,LAL vs. GSW,L,48.0,45,95,0.474,15,42,0.357,9,19,0.474,5,40,45,21,18.0,7,4,2,25,18,114,-7.0
1335,2021-22,1610612744,GSW,Golden State Warriors,0022100002,2021-10-19T00:00:00,GSW @ LAL,W,48.0,41,93,0.441,14,39,0.359,25,30,0.833,9,41,50,30,17.0,9,2,4,18,25,121,7.0


In [5]:
g_columns = ['SEASON_YEAR', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'MIN']
h_columns = ['TEAM_ID_(H)', 'TEAM_ABBREVIATION_(H)', 'TEAM_NAME_(H)', 'WL_(H)', 'FGM_(H)',
             'FGA_(H)', 'FG_PCT_(H)', 'FG3M_(H)', 'FG3A_(H)', 'FG3_PCT_(H)', 'FTM_(H)', 'FTA_(H)',
             'FT_PCT_(H)', 'OREB_(H)', 'DREB_(H)', 'REB_(H)', 'AST_(H)','TOV_(H)', 'STL_(H)', 'BLK_(H)',
             'BLKA_(H)', 'PF_(H)', 'PFD_(H)', 'PTS_(H)']
a_columns = ['TEAM_ID_(A)', 'TEAM_ABBREVIATION_(A)', 'TEAM_NAME_(A)', 'WL_(A)', 'FGM_(A)',
             'FGA_(A)', 'FG_PCT_(A)', 'FG3M_(A)', 'FG3A_(A)', 'FG3_PCT_(A)', 'FTM_(A)', 'FTA_(A)',
             'FT_PCT_(A)', 'OREB_(A)', 'DREB_(A)', 'REB_(A)', 'AST_(A)','TOV_(A)', 'STL_(A)', 'BLK_(A)',
             'BLKA_(A)', 'PF_(A)', 'PFD_(A)', 'PTS_(A)']
df = pd.DataFrame(columns=[g_columns+h_columns+a_columns+['DIFFERENCE']], index=[logs['GAME_ID'].unique()])

In [6]:
for i, row in tqdm(logs.iterrows()):
    if row['MATCHUP'].find('@') == -1:
        for col in row.index:
            if col in g_columns:
                df.at[row['GAME_ID'], col] = row[col]
            elif (col+'_(H)') in h_columns:
                df.at[row['GAME_ID'], (col+'_(H)')] = row[col]
    else:
        for col in row.index:
            if (col+'_(A)') in a_columns:
                df.at[row['GAME_ID'], (col+'_(A)')] = row[col]
for i, row in tqdm(df.iterrows()):
    df.at[i, 'DIFFERENCE'] = abs(row['PTS_(H)'] - row['PTS_(A)'])
df.reset_index()

15454it [03:11, 80.56it/s]
7727it [00:03, 1937.64it/s]


,level_0,SEASON_YEAR,GAME_ID,GAME_DATE,MATCHUP,MIN,TEAM_ID_(H),TEAM_ABBREVIATION_(H),TEAM_NAME_(H),WL_(H),FGM_(H),FGA_(H),FG_PCT_(H),FG3M_(H),FG3A_(H),FG3_PCT_(H),FTM_(H),FTA_(H),FT_PCT_(H),OREB_(H),DREB_(H),REB_(H),AST_(H),TOV_(H),STL_(H),BLK_(H),BLKA_(H),PF_(H),PFD_(H),PTS_(H),TEAM_ID_(A),TEAM_ABBREVIATION_(A),TEAM_NAME_(A),WL_(A),FGM_(A),FGA_(A),FG_PCT_(A),FG3M_(A),FG3A_(A),FG3_PCT_(A),FTM_(A),FTA_(A),FT_PCT_(A),OREB_(A),DREB_(A),REB_(A),AST_(A),TOV_(A),STL_(A),BLK_(A),BLKA_(A),PF_(A),PFD_(A),PTS_(A),DIFFERENCE
0,0021501221,2015-16,0021501221,2016-04-13T00:00:00,WAS vs. ATL,48.0,1610612764,WAS,Washington Wizards,W,42,88,0.477,9,28,0.321,16,30,0.533,11,40,51,25,19.0,14,4,5,22,21,109,1610612737,ATL,Atlanta Hawks,L,32,81,0.395,11,30,0.367,23,31,0.742,9,38,47,22,23.0,13,5,4,21,22,98,11
1,0021501227,2015-16,0021501227,2016-04-13T00:00:00,GSW vs. MEM,48.0,1610612744,GSW,Golden State Warriors,W,46,87,0.529,20,47,0.426,13,16,0.813,12,39,51,35,17.0,7,7,1,14,18,125,1610612763,MEM,Memphis Grizzlies,L,42,98,0.429,8,24,0.333,12,16,0.75,13,27,40,25,13.0,12,1,7,18,14,104,21
2,0021501223,2015-16,0021501223,2016-04-13T00:00:00,DAL vs. SAS,48.0,1610612742,DAL,Dallas Mavericks,L,31,77,0.403,12,39,0.308,17,21,0.81,11,30,41,20,12.0,8,2,3,23,19,91,1610612759,SAS,San Antonio Spurs,W,35,74,0.473,6,23,0.261,20,26,0.769,8,34,42,24,15.0,8,3,2,19,23,96,5
3,0021501222,2015-16,0021501222,2016-04-13T00:00:00,CHI vs. PHI,48.0,1610612741,CHI,Chicago Bulls,W,39,83,0.47,15,24,0.625,22,27,0.815,16,37,53,27,17.0,8,8,7,13,24,115,1610612755,PHI,Philadelphia 76ers,L,37,81,0.457,17,35,0.486,14,18,0.778,4,27,31,28,13.0,10,7,8,24,13,105,10
4,0021501217,2015-16,0021501217,2016-04-13T00:00:00,BOS vs. MIA,48.0,1610612738,BOS,Boston Celtics,W,37,86,0.43,5,14,0.357,19,22,0.864,10,29,39,20,7.0,7,3,10,20,21,98,1610612748,MIA,Miami Heat,L,38,88,0.432,2,19,0.105,10,12,0.833,14,30,44,14,10.0,5,10,3,21,20,88,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7722,0022100010,2021-22,0022100010,2021-10-20T00:00:00,SAS vs. ORL,48.0,1610612759,SAS,San Antonio Spurs,W,49,98,0.5,13,30,0.433,12,18,0.667,13,30,43,32,8.0,8,3,6,22,20,123,1610612753,ORL,Orlando Magic,L,35,86,0.407,9,29,0.31,18,23,0.783,13,31,44,17,16.0,6,6,3,20,22,97,26
7723,0022100007,2021-22,0022100007,2021-10-20T00:00:00,MEM vs. CLE,48.0,1610612763,MEM,Memphis Grizzlies,W,53,100,0.53,14,33,0.424,12,12,1.0,13,40,53,28,12.0,8,8,5,15,17,132,1610612739,CLE,Cleveland Cavaliers,L,47,93,0.505,14,38,0.368,13,18,0.722,7,29,36,38,11.0,6,5,8,17,15,121,11
7724,0022100004,2021-22,0022100004,2021-10-20T00:00:00,DET vs. CHI,48.0,1610612765,DET,Detroit Pistons,L,36,90,0.4,6,28,0.214,10,13,0.769,11,36,47,17,17.0,7,5,5,16,19,88,1610612741,CHI,Chicago Bulls,W,37,86,0.43,7,23,0.304,13,15,0.867,9,39,48,18,17.0,8,5,5,19,16,94,6
7725,0022100001,2021-22,0022100001,2021-10-19T00:00:00,MIL vs. BKN,48.0,1610612749,MIL,Milwaukee Bucks,W,48,105,0.457,17,45,0.378,14,18,0.778,13,41,54,25,8.0,8,9,9,19,17,127,1610612751,BKN,Brooklyn Nets,L,37,84,0.44,17,32,0.531,13,23,0.565,5,39,44,19,13.0,3,9,9,17,19,104,23


In [8]:
df.to_csv('game_logs.csv')